This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker import NNNodeBenchmarkerSSL
from graph_world.models.basic_gnn import GCN, SuperGAT
from torch_geometric.datasets import Planetoid, KarateClub, FakeDataset
from torch_geometric.transforms import RandomNodeSplit

from graph_world.self_supervised_learning.pretext_tasks.auxiliary_property_based import *


In [2]:
# Get dataset
dataset = FakeDataset(num_graphs = 1, avg_num_nodes = 100, num_channels = 16, num_classes = 4)[0]
dataset = RandomNodeSplit(split = "random", num_test = 20, num_val = 20, num_train_per_class=2)(dataset)

In [7]:
# Parameter setup (for cora)
benchmark_params = {
    'downstream_epochs' : 200,
    'pretext_epochs' : 200,
    'downstream_lr' : 3e-4,
    'pretext_lr' : 3e-4,
    'patience': 100
}

h_params = {
    'in_channels' : dataset.x.shape[1],
    'hidden_channels' : 128,
    'num_layers' : 2,
    'dropout' : 0.5,
}

pretext_params = {
    'k_largest': 3,
}

generator_config = {
    'num_clusters' : 4,
}

pretext_task = PairwiseAttrSim
training_scheme = 'JL'

In [8]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerSSL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params,
                pretext_task=pretext_task, pretext_params = pretext_params, training_scheme=training_scheme)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

TypeError: PairwiseAttrSim.__init__() missing 1 required positional argument: 'k_largest'
  No values supplied by Gin or caller for arguments: ['k_largest']
  Gin had values bound for: []
  Caller supplied values for: ['ae_loss_weight', 'data', 'embedding_mask_ratio', 'encoder', 'epochs', 'er_loss_weight', 'feature_mask_ratio', 'fr_loss_weight', 'partial_reconstruction', 'pretext_weight', 'self', 'train_mask']
  In call to configurable 'PairwiseAttrSim' (<class 'graph_world.self_supervised_learning.pretext_tasks.auxiliary_property_based.PairwiseAttrSim'>)